In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.ensemble import RandomForestClassifier

In [2]:
# ===============================================================
# (1) Load Data
# ===============================================================
players = pd.read_csv("data/players.csv")
players_teams = pd.read_csv("data/players_teams.csv")
awards = pd.read_csv("data/awards_players.csv")
teams = pd.read_csv("data/teams.csv")
teams_post = pd.read_csv("data/teams_post.csv")

# Create output directory
os.makedirs("c", exist_ok=True)
MODEL_DIR = "c/"

print(f"Players: {players.shape}")
print(f"Players_teams: {players_teams.shape}")
print(f"Awards: {awards.shape}")
print(f"Teams: {teams.shape}")
print(f"Teams_post: {teams_post.shape}")

Players: (893, 10)
Players_teams: (1876, 43)
Awards: (95, 4)
Teams: (142, 61)
Teams_post: (80, 5)


In [3]:
# ===============================================================
# (2) Define main awards
# ===============================================================
main_awards = [
    'Most Valuable Player',
    'Defensive Player of the Year',
    'Most Improved Player',
    'WNBA Finals Most Valuable Player',
    'Rookie of the Year',
    'All-Star Game Most Valuable Player'
]

# Coach award is separate (different entity type)
coach_award = 'Coach of the Year'

awards_main = awards[awards['award'].isin(main_awards)]
awards_coach = awards[awards['award'] == coach_award]

max_year = players_teams['year'].max()
print(f"Data spans years 1-{max_year}")

print(f"\nPlayer Awards distribution:")
for award in main_awards:
    count = len(awards_main[awards_main['award'] == award])
    print(f"  {award}: {count}")

print(f"\nCoach Award:")
print(f"  {coach_award}: {len(awards_coach)}")

Data spans years 1-10

Player Awards distribution:
  Most Valuable Player: 10
  Defensive Player of the Year: 10
  Most Improved Player: 9
  WNBA Finals Most Valuable Player: 10
  Rookie of the Year: 10
  All-Star Game Most Valuable Player: 8

Coach Award:
  Coach of the Year: 10


In [ ]:
# ===============================================================
# (3) Calculate player performance stats by year
# ===============================================================
player_stats = players_teams.copy()
player_stats = player_stats[player_stats['GP'] >= 10]

# Per-game statistics
player_stats['ppg'] = player_stats['points'] / player_stats['GP']
player_stats['rpg'] = player_stats['rebounds'] / player_stats['GP']
player_stats['apg'] = player_stats['assists'] / player_stats['GP']
player_stats['spg'] = player_stats['steals'] / player_stats['GP']
player_stats['bpg'] = player_stats['blocks'] / player_stats['GP']
player_stats['mpg'] = player_stats['minutes'] / player_stats['GP']
player_stats['topg'] = player_stats['turnovers'] / player_stats['GP']
player_stats['fg_pct'] = np.where(player_stats['fgAttempted'] > 0, 
                                   player_stats['fgMade'] / player_stats['fgAttempted'], 0)

# Efficiency rating
player_stats['efficiency'] = (player_stats['points'] + player_stats['rebounds'] + 
                               player_stats['assists'] + player_stats['steals'] + 
                               player_stats['blocks'] - player_stats['turnovers']) / player_stats['GP']

player_stats['player_rating'] = (
    player_stats['points'] +
    player_stats['rebounds'] * 1.2 +
    player_stats['assists'] * 1.5 +
    player_stats['steals'] * 3 +
    player_stats['blocks'] * 3 -
    player_stats['turnovers'] * 2 -
    player_stats['PF'] * 0.5
) / (player_stats['minutes'] / 36 + 1)

player_stats['gs_pct'] = player_stats['GS'] / player_stats['GP']

# Defensive-specific stats
player_stats['drpg'] = player_stats['dRebounds'] / player_stats['GP']  # defensive rebounds per game
player_stats['def_rating'] = (player_stats['steals'] * 2 + player_stats['blocks'] * 3 + player_stats['dRebounds']) / player_stats['GP']

print(f"Player stats calculated: {player_stats.shape}")

Player stats calculated: (1603, 56)


In [5]:
# ===============================================================
# (4) Calculate league averages by year
# ===============================================================
yearly_avg = player_stats.groupby('year').agg({
    'ppg': 'mean', 'rpg': 'mean', 'apg': 'mean', 'spg': 'mean', 'bpg': 'mean',
    'efficiency': 'mean', 'mpg': 'mean', 'player_rating': 'mean'
}).reset_index()
yearly_avg.columns = ['year', 'league_ppg', 'league_rpg', 'league_apg', 'league_spg', 
                      'league_bpg', 'league_efficiency', 'league_mpg', 'league_player_rating']

# Elite cutoff (top 10%)
elite_cutoffs = player_stats.groupby('year')['player_rating'].quantile(0.90).reset_index()
elite_cutoffs.columns = ['year', 'elite_cutoff']

player_stats = player_stats.merge(yearly_avg, on='year', how='left')
player_stats = player_stats.merge(elite_cutoffs, on='year', how='left')
player_stats['is_elite'] = (player_stats['player_rating'] >= player_stats['elite_cutoff']).astype(int)

# Save for test time
yearly_avg.to_csv("c/league_averages.csv", index=False)
elite_cutoffs.to_csv("c/elite_cutoffs.csv", index=False)
print("League averages saved")

League averages saved


In [6]:
# ===============================================================
# (5) Add team success context
# ===============================================================
teams_success = teams[['year', 'tmID', 'won', 'lost', 'confID', 'playoff', 'o_pts', 'd_pts', 'GP']].copy()
teams_success['win_pct'] = teams_success['won'] / (teams_success['won'] + teams_success['lost'])
teams_success['conf_rank'] = teams_success.groupby(['year', 'confID'])['win_pct'].rank(ascending=False)
teams_success['made_playoffs'] = teams_success['playoff'].notna().astype(int)
teams_success['margin_per_game'] = (teams_success['o_pts'] - teams_success['d_pts']) / teams_success['GP']

# Championship indicator
playoff_wins = teams_post.groupby(['year', 'tmID'])['W'].sum().reset_index()
champion_idx = playoff_wins.groupby('year')['W'].idxmax()
champions = playoff_wins.loc[champion_idx][['year', 'tmID']].copy()
champions['won_championship'] = 1
teams_success = teams_success.merge(champions, on=['year', 'tmID'], how='left')
teams_success['won_championship'] = teams_success['won_championship'].fillna(0).astype(int)

# Playoff wins
playoff_stats = teams_post.groupby(['year', 'tmID']).agg({'W': 'sum'}).reset_index()
playoff_stats.columns = ['year', 'tmID', 'playoff_wins']
teams_success = teams_success.merge(playoff_stats, on=['year', 'tmID'], how='left')
teams_success['playoff_wins'] = teams_success['playoff_wins'].fillna(0).astype(int)

print(f"Teams success calculated: {teams_success.shape}")

Teams success calculated: (142, 15)


In [7]:
# ===============================================================
# (6) Build Training Dataset
# ===============================================================

# predict awards in year Y using features from year Y-1
training_records = []
years = sorted(player_stats['year'].unique())
print(f"Building training data for years {years[1]}-{years[-1]}")

for target_year in years[1:]:  # start from year 2
    prev_year = target_year - 1
    
    prev_year_stats = player_stats[player_stats['year'] == prev_year].copy()
    prev_year_teams = teams_success[teams_success['year'] == prev_year].copy()
    target_year_awards = awards_main[awards_main['year'] == target_year].copy()
    
    for _, player_row in prev_year_stats.iterrows():
        player_id = player_row['playerID']
        tm_id = player_row['tmID']
        
        team_row = prev_year_teams[prev_year_teams['tmID'] == tm_id]
        
        record = {
            'playerID': player_id,
            'tmID': tm_id,
            'target_year': target_year,
            'prev_year': prev_year,
            
            # T-1 player stats
            'ppg_prev': player_row['ppg'],
            'rpg_prev': player_row['rpg'],
            'apg_prev': player_row['apg'],
            'spg_prev': player_row['spg'],
            'bpg_prev': player_row['bpg'],
            'mpg_prev': player_row['mpg'],
            'fg_pct_prev': player_row['fg_pct'],
            'efficiency_prev': player_row['efficiency'],
            'player_rating_prev': player_row['player_rating'],
            'is_elite_prev': player_row['is_elite'],
            'GP_prev': player_row['GP'],
            'GS_prev': player_row['GS'],
            'gs_pct_prev': player_row['gs_pct'],
            'minutes_prev': player_row['minutes'],
            
            # Comparison to league
            'ppg_vs_league': player_row['ppg'] - player_row['league_ppg'],
            'rpg_vs_league': player_row['rpg'] - player_row['league_rpg'],
            'apg_vs_league': player_row['apg'] - player_row['league_apg'],
            'efficiency_vs_league': player_row['efficiency'] - player_row['league_efficiency'],
            'rating_vs_league': player_row['player_rating'] - player_row['league_player_rating'],
            
            # Defensive-specific stats
            'drpg_prev': player_row['drpg'],
            'def_rating_prev': player_row['def_rating'],
        }
        
        # Team context
        if len(team_row) > 0:
            team_row = team_row.iloc[0]
            record['win_pct_prev'] = team_row['win_pct']
            record['conf_rank_prev'] = team_row['conf_rank']
            record['made_playoffs_prev'] = team_row['made_playoffs']
            record['margin_prev'] = team_row['margin_per_game']
            record['won_championship_prev'] = team_row['won_championship']
            record['playoff_wins_prev'] = team_row['playoff_wins']
        else:
            record['win_pct_prev'] = 0.5
            record['conf_rank_prev'] = 6
            record['made_playoffs_prev'] = 0
            record['margin_prev'] = 0
            record['won_championship_prev'] = 0
            record['playoff_wins_prev'] = 0
        
        # Award targets
        for award in main_awards:
            col_name = f"won_{award.replace(' ', '_').lower()}"
            won = len(target_year_awards[
                (target_year_awards['playerID'] == player_id) & 
                (target_year_awards['award'] == award)
            ]) > 0
            record[col_name] = int(won)
        
        training_records.append(record)

train_df = pd.DataFrame(training_records)
print(f"\nTraining dataset shape: {train_df.shape}")

Building training data for years 2-10

Training dataset shape: (1458, 37)


In [8]:
# ===============================================================
# (7) Add career features
# ===============================================================
player_first_year = player_stats.groupby('playerID')['year'].min().reset_index()
player_first_year.columns = ['playerID', 'first_year']
train_df = train_df.merge(player_first_year, on='playerID', how='left')
train_df['years_experience'] = train_df['prev_year'] - train_df['first_year']

# Career awards
for award in main_awards:
    award_col = f"career_{award.replace(' ', '_').lower()}"
    train_df[award_col] = 0
train_df['career_total_awards'] = 0

for idx, row in train_df.iterrows():
    player_id = row['playerID']
    target_year = row['target_year']
    
    past_awards = awards_main[
        (awards_main['playerID'] == player_id) & 
        (awards_main['year'] < target_year)
    ]
    
    total = 0
    for award in main_awards:
        award_col = f"career_{award.replace(' ', '_').lower()}"
        count = len(past_awards[past_awards['award'] == award])
        train_df.at[idx, award_col] = count
        total += count
    train_df.at[idx, 'career_total_awards'] = total

print(f"Career features added")
print(f"\nAward counts in training:")
for award in main_awards:
    col = f"won_{award.replace(' ', '_').lower()}"
    print(f"  {award}: {train_df[col].sum()}")

Career features added

Award counts in training:
  Most Valuable Player: 7
  Defensive Player of the Year: 7
  Most Improved Player: 8
  WNBA Finals Most Valuable Player: 9
  Rookie of the Year: 0
  All-Star Game Most Valuable Player: 7


In [9]:
# ===============================================================
# (8) Add improvement features (T-1 vs T-2)
# ===============================================================
player_stats_sorted = player_stats.sort_values(['playerID', 'year'])
player_stats_sorted['ppg_prev2'] = player_stats_sorted.groupby('playerID')['ppg'].shift(1)
player_stats_sorted['efficiency_prev2'] = player_stats_sorted.groupby('playerID')['efficiency'].shift(1)
player_stats_sorted['player_rating_prev2'] = player_stats_sorted.groupby('playerID')['player_rating'].shift(1)
player_stats_sorted['mpg_prev2'] = player_stats_sorted.groupby('playerID')['mpg'].shift(1)

prev2_cols = ['playerID', 'year', 'ppg_prev2', 'efficiency_prev2', 'player_rating_prev2', 'mpg_prev2']
prev2_data = player_stats_sorted[prev2_cols].copy()
prev2_data = prev2_data.rename(columns={'year': 'prev_year'})

train_df = train_df.merge(prev2_data, on=['playerID', 'prev_year'], how='left')

# Calculate improvement
train_df['ppg_improvement'] = train_df['ppg_prev'] - train_df['ppg_prev2'].fillna(train_df['ppg_prev'])
train_df['efficiency_improvement'] = train_df['efficiency_prev'] - train_df['efficiency_prev2'].fillna(train_df['efficiency_prev'])
train_df['rating_improvement'] = train_df['player_rating_prev'] - train_df['player_rating_prev2'].fillna(train_df['player_rating_prev'])
train_df['mpg_improvement'] = train_df['mpg_prev'] - train_df['mpg_prev2'].fillna(train_df['mpg_prev'])

print(f"Improvement features added")

Improvement features added


In [10]:
# ===============================================================
# (9) Add playoff performance features
# ===============================================================
playoff_player_cols = ['playerID', 'year', 'tmID', 'PostGP', 'PostGS', 'PostMinutes', 
                       'PostPoints', 'PostRebounds', 'PostAssists', 'PostSteals', 'PostBlocks']

existing_cols = [c for c in playoff_player_cols if c in players_teams.columns]
playoff_data = players_teams[existing_cols].copy()
playoff_data = playoff_data.rename(columns={'year': 'prev_year'})

rename_dict = {c: f"{c}_prev" for c in existing_cols if c not in ['playerID', 'prev_year', 'tmID']}
playoff_data = playoff_data.rename(columns=rename_dict)

train_df = train_df.merge(playoff_data, on=['playerID', 'prev_year', 'tmID'], how='left')

for col in train_df.columns:
    if 'Post' in col:
        train_df[col] = train_df[col].fillna(0)

# Calculate per-game playoff stats
if 'PostGP_prev' in train_df.columns and 'PostPoints_prev' in train_df.columns:
    train_df['playoff_ppg_prev'] = np.where(
        train_df['PostGP_prev'] > 0,
        train_df['PostPoints_prev'] / train_df['PostGP_prev'],
        0
    )

print(f"Playoff features added")
print(f"Final training shape: {train_df.shape}")

Playoff features added
Final training shape: (1502, 63)


In [11]:
# ===============================================================
# (9.5) Build ROY Training Dataset
# ===============================================================

def build_roy_dataset(start_year, end_year):
    records = []
    years = range(start_year, end_year + 1)
    
    print(f"\nBuilding ROY dataset for years {start_year}-{end_year}")
    
    coaches = pd.read_csv("data/coaches.csv")
    
    # get first year for each player to identify rookies
    player_first_year = players_teams.groupby('playerID')['year'].min().reset_index()
    player_first_year.columns = ['playerID', 'first_year']
    
    for target_year in years:
        # calculate college scores
        past_roy_winners = awards[(awards['award'] == 'Rookie of the Year') & (awards['year'] < target_year)]['playerID'].tolist()
        past_roy_colleges = players[players['bioID'].isin(past_roy_winners)]['college'].value_counts()
        
        # create college reputation score
        all_colleges = players['college'].unique()
        college_scores = {}
        for college in all_colleges:
            if college in past_roy_colleges.index:
                college_scores[college] = past_roy_colleges[college] + 1
            elif college and college != 'none' and college != '':
                college_scores[college] = 1
            else:
                college_scores[college] = 0
        
        # calculate coach history using ONLY data from years BEFORE target_year
        coaches_before_target = coaches[coaches['year'] < target_year]
        coach_history = coaches_before_target.groupby('coachID').agg({
            'won': 'sum',
            'lost': 'sum',
            'post_wins': 'sum',
            'post_losses': 'sum'
        }).reset_index()
        coach_history['coach_career_win_pct'] = coach_history['won'] / (coach_history['won'] + coach_history['lost'] + 0.001)
        coach_history['coach_career_playoff_wins'] = coach_history['post_wins']
        
        # get target year roster
        target_roster = players_teams[players_teams['year'] == target_year][['playerID', 'tmID']].drop_duplicates()
        
        # identify rookies
        target_roster = target_roster.merge(player_first_year, on='playerID', how='left')
        rookies = target_roster[target_roster['first_year'] == target_year].copy()
        
        if len(rookies) == 0:
            continue
        
        # get target year awards
        target_year_awards = awards_main[awards_main['year'] == target_year].copy()
        
        # get team data from previous year (pre-season info)
        prev_year_teams = teams_success[teams_success['year'] == target_year - 1].copy()
        
        # get coach for each team in target year
        target_coaches = coaches[coaches['year'] == target_year][['tmID', 'coachID']].drop_duplicates()
        
        for _, rookie_row in rookies.iterrows():
            player_id = rookie_row['playerID']
            tm_id = rookie_row['tmID']
            
            # get player biometric data
            player_bio = players[players['bioID'] == player_id]
            
            if len(player_bio) == 0:
                continue
            
            player_bio = player_bio.iloc[0]
            
            # get position encoding
            pos = str(player_bio['pos']) if pd.notna(player_bio['pos']) else ''
            is_guard = 1 if 'G' in pos else 0
            is_forward = 1 if 'F' in pos else 0
            is_center = 1 if 'C' in pos else 0
            
            # get college info
            college = player_bio['college'] if pd.notna(player_bio['college']) else ''
            college_score = college_scores.get(college, 0)
            has_college = 1 if college and college != 'none' and college != '' else 0
            
            # get coach info
            team_coach = target_coaches[target_coaches['tmID'] == tm_id]
            if len(team_coach) > 0:
                coach_id = team_coach.iloc[0]['coachID']
                coach_stats = coach_history[coach_history['coachID'] == coach_id]
                if len(coach_stats) > 0:
                    coach_win_pct = coach_stats.iloc[0]['coach_career_win_pct']
                    coach_playoff_wins = coach_stats.iloc[0]['coach_career_playoff_wins']
                else:
                    coach_win_pct = 0.5
                    coach_playoff_wins = 0
            else:
                coach_win_pct = 0.5
                coach_playoff_wins = 0
            
            # get team context from previous year
            team_row = prev_year_teams[prev_year_teams['tmID'] == tm_id]
            if len(team_row) > 0:
                team_row = team_row.iloc[0]
                team_win_pct = team_row['win_pct']
                team_conf_rank = team_row['conf_rank']
                team_made_playoffs = team_row['made_playoffs']
            else:
                team_win_pct = 0.5
                team_conf_rank = 6
                team_made_playoffs = 0
            
            record = {
                'playerID': player_id,
                'tmID': tm_id,
                'target_year': target_year,
                
                # biometric features
                'height': player_bio['height'] if pd.notna(player_bio['height']) else 70,
                'weight': player_bio['weight'] if pd.notna(player_bio['weight']) else 160,
                
                # position
                'is_guard': is_guard,
                'is_forward': is_forward,
                'is_center': is_center,
                
                # college features
                'college_score': college_score,
                'has_college': has_college,
                
                # coach features (historical, before target year)
                'coach_win_pct': coach_win_pct,
                'coach_playoff_wins': coach_playoff_wins,
                
                # team context (from previous year)
                'team_win_pct': team_win_pct,
                'team_conf_rank': team_conf_rank,
                'team_made_playoffs': team_made_playoffs,
                
                # target
                'won_rookie_of_the_year': int(len(target_year_awards[
                    (target_year_awards['playerID'] == player_id) & 
                    (target_year_awards['award'] == 'Rookie of the Year')
                ]) > 0)
            }
            
            records.append(record)
    
    df = pd.DataFrame(records)
    print(f"ROY Dataset built: {df.shape} (rookies only)")
    return df

roy_train_df = build_roy_dataset(1, max_year)
roy_train_df.to_csv("c/roy_train_data.csv", index=False)
print(f"ROY training data saved: c/roy_train_data.csv")
print(f"ROY winners in training: {roy_train_df['won_rookie_of_the_year'].sum()}")


Building ROY dataset for years 1-10
ROY Dataset built: (564, 16) (rookies only)
ROY training data saved: c/roy_train_data.csv
ROY winners in training: 10


In [12]:
# ===============================================================
# (9.6) Build COTY Training Dataset
# ===============================================================

def build_coty_dataset(start_year, end_year):
    records = []
    years = range(start_year, end_year + 1)
    
    print(f"\nBuilding COTY dataset for years {start_year}-{end_year}")
    
    coaches = pd.read_csv("data/coaches.csv")
    
    # Build teams_success for all years
    teams_success_all = teams[['year', 'tmID', 'won', 'lost', 'confID', 'playoff', 'o_pts', 'd_pts', 'GP']].copy()
    teams_success_all['win_pct'] = teams_success_all['won'] / (teams_success_all['won'] + teams_success_all['lost'])
    teams_success_all['conf_rank'] = teams_success_all.groupby(['year', 'confID'])['win_pct'].rank(ascending=False)
    teams_success_all['made_playoffs'] = teams_success_all['playoff'].notna().astype(int)
    teams_success_all['playoff_wins'] = 0  # Will compute from post data if available
    teams_success_all['margin_per_game'] = (teams_success_all['o_pts'] - teams_success_all['d_pts']) / teams_success_all['GP']
    
    for target_year in years:
        prev_year = target_year - 1
        
        # Get coaches for target year (to know who is coaching which team)
        year_coaches = coaches[coaches['year'] == target_year].copy()
        
        if len(year_coaches) == 0:
            continue
        
        # Get T-1 team records (what we can see before the season)
        prev_year_teams = teams_success_all[teams_success_all['year'] == prev_year].copy()
        
        # Get T-2 team records for improvement calculation
        prev2_year_teams = teams_success_all[teams_success_all['year'] == prev_year - 1].copy()
        
        # Calculate coach career history using ONLY years BEFORE target_year
        coaches_before_target = coaches[coaches['year'] < target_year]
        coach_career = coaches_before_target.groupby('coachID').agg({
            'won': 'sum',
            'lost': 'sum',
            'post_wins': 'sum',
            'year': 'nunique'  # years of experience
        }).reset_index()
        coach_career.columns = ['coachID', 'career_wins', 'career_losses', 'career_playoff_wins', 'years_coaching']
        coach_career['career_win_pct'] = coach_career['career_wins'] / (coach_career['career_wins'] + coach_career['career_losses'] + 0.001)
        
        # Check if coach won COTY before
        past_coty = awards_coach[awards_coach['year'] < target_year]['playerID'].tolist()
        
        # Get COTY winner for this year
        coty_winner = awards_coach[awards_coach['year'] == target_year]
        coty_winner_id = coty_winner['playerID'].values[0] if len(coty_winner) > 0 else None
        
        for _, coach_row in year_coaches.iterrows():
            coach_id = coach_row['coachID']
            tm_id = coach_row['tmID']
            
            # T-1 team performance (previous season results)
            prev_team = prev_year_teams[prev_year_teams['tmID'] == tm_id]
            if len(prev_team) > 0:
                prev_team = prev_team.iloc[0]
                wins_prev = prev_team['won'] if 'won' in prev_team else 0
                win_pct_prev = prev_team['win_pct']
                conf_rank_prev = prev_team['conf_rank']
                made_playoffs_prev = prev_team['made_playoffs']
                playoff_wins_prev = coach_row.get('post_wins', 0)  # Use coach's prev post wins
                margin_prev = prev_team['margin_per_game']
            else:
                wins_prev = 0
                win_pct_prev = 0.5
                conf_rank_prev = 6
                made_playoffs_prev = 0
                playoff_wins_prev = 0
                margin_prev = 0
            
            # T-1 vs T-2 improvement (team trajectory before season starts)
            prev2_team = prev2_year_teams[prev2_year_teams['tmID'] == tm_id]
            if len(prev2_team) > 0:
                prev2_team = prev2_team.iloc[0]
                prev2_win_pct = prev2_team['win_pct']
                prev2_made_playoffs = prev2_team['made_playoffs']
                win_improvement_prev = win_pct_prev - prev2_win_pct
                playoff_improvement_prev = made_playoffs_prev - prev2_made_playoffs
            else:
                win_improvement_prev = 0
                playoff_improvement_prev = 0
            
            # Coach career history (before target year)
            coach_hist = coach_career[coach_career['coachID'] == coach_id]
            if len(coach_hist) > 0:
                career_win_pct = coach_hist.iloc[0]['career_win_pct']
                career_playoff_wins = coach_hist.iloc[0]['career_playoff_wins']
                years_coaching = coach_hist.iloc[0]['years_coaching']
            else:
                career_win_pct = 0.5
                career_playoff_wins = 0
                years_coaching = 0
            
            # Has won COTY before
            won_coty_before = 1 if coach_id in past_coty else 0
            
            record = {
                'coachID': coach_id,
                'tmID': tm_id,
                'target_year': target_year,
                
                # T-1 team performance (previous season)
                'wins_prev': wins_prev,
                'win_pct_prev': win_pct_prev,
                'conf_rank_prev': conf_rank_prev,
                'made_playoffs_prev': made_playoffs_prev,
                'playoff_wins_prev': playoff_wins_prev,
                'margin_prev': margin_prev,
                
                # Team trajectory (T-1 vs T-2)
                'win_improvement_prev': win_improvement_prev,
                'playoff_improvement_prev': playoff_improvement_prev,
                
                # Coach career history (before target year)
                'career_win_pct': career_win_pct,
                'career_playoff_wins': career_playoff_wins,
                'years_coaching': years_coaching,
                'won_coty_before': won_coty_before,
                
                # Target
                'won_coach_of_the_year': int(coach_id == coty_winner_id)
            }
            
            records.append(record)
    
    df = pd.DataFrame(records)
    print(f"COTY Dataset built: {df.shape}")
    return df

coty_train_df = build_coty_dataset(2, max_year)
coty_train_df.to_csv("c/coty_train_data.csv", index=False)
print(f"COTY training data saved: c/coty_train_data.csv")
print(f"COTY winners in training: {coty_train_df['won_coach_of_the_year'].sum()}")


Building COTY dataset for years 2-10
COTY Dataset built: (145, 16)
COTY training data saved: c/coty_train_data.csv
COTY winners in training: 9


In [13]:
# ===============================================================
# (10) Save training data
# ===============================================================
train_df.to_csv("c/award_train_data.csv", index=False)
print(f"\nTraining data saved: c/award_train_data.csv")


Training data saved: c/award_train_data.csv


In [14]:
# ===============================================================
# (11) Define feature sets
# ===============================================================

# MVP Features
features_mvp = [
    'ppg_prev', 'rpg_prev', 'apg_prev', 'spg_prev', 'bpg_prev', 'mpg_prev',
    'fg_pct_prev', 'efficiency_prev', 'player_rating_prev', 'gs_pct_prev',
    'GP_prev', 'minutes_prev',
    'ppg_vs_league', 'rpg_vs_league', 'apg_vs_league',
    'efficiency_vs_league', 'rating_vs_league',
    'is_elite_prev',
    'win_pct_prev', 'conf_rank_prev', 'made_playoffs_prev', 'margin_prev',
    'years_experience',
    'career_total_awards', 'career_most_valuable_player'
]

# DPOY Features
features_dpoy = [
    'spg_prev', 'bpg_prev', 'drpg_prev', 'def_rating_prev',
    'rpg_prev', 'mpg_prev', 'GP_prev', 'gs_pct_prev',
    'efficiency_prev', 'player_rating_prev',
    'win_pct_prev', 'made_playoffs_prev',
    'career_defensive_player_of_the_year',
    'years_experience', 'career_total_awards'
]

# MIP Features
features_mip = [
    'ppg_prev', 'efficiency_prev', 'player_rating_prev', 'mpg_prev',
    'GP_prev', 'gs_pct_prev',
    'ppg_improvement', 'efficiency_improvement', 'rating_improvement', 'mpg_improvement',
    'ppg_vs_league', 'efficiency_vs_league',
    'win_pct_prev',
    'years_experience',
    'career_total_awards'
]

# FMVP Features
features_fmvp = [
    'PostPoints_prev', 'PostRebounds_prev', 'PostAssists_prev',
    'PostSteals_prev', 'PostBlocks_prev',
    'PostGP_prev', 'PostGS_prev', 'PostMinutes_prev',
    'playoff_ppg_prev',
    'player_rating_prev', 'efficiency_prev',
    'ppg_prev', 'rpg_prev', 'apg_prev',
    'won_championship_prev', 'playoff_wins_prev',
    'years_experience',
    'career_total_awards', 'career_wnba_finals_most_valuable_player'
]

# ROY Features
features_roy = [
    'height', 'weight',
    'is_guard', 'is_forward', 'is_center',
    'college_score', 'has_college',
    'coach_win_pct', 'coach_playoff_wins',
    'team_win_pct', 'team_conf_rank', 'team_made_playoffs'
]

# COTY Features
features_coty = [
    'wins_prev', 'win_pct_prev', 'conf_rank_prev', 'made_playoffs_prev', 'playoff_wins_prev', 'margin_prev',
    'win_improvement_prev', 'playoff_improvement_prev',
    'career_win_pct', 'career_playoff_wins', 'years_coaching', 'won_coty_before'
]

# ASG_MVP Features (elite players + star power)
features_asg_mvp = [
    'ppg_prev', 'rpg_prev', 'apg_prev', 'spg_prev', 'bpg_prev', 'mpg_prev',
    'fg_pct_prev', 'efficiency_prev', 'player_rating_prev', 'gs_pct_prev',
    'GP_prev', 'minutes_prev',
    'ppg_vs_league', 'rpg_vs_league', 'apg_vs_league',
    'efficiency_vs_league', 'rating_vs_league',
    'is_elite_prev',
    'win_pct_prev', 'conf_rank_prev', 'made_playoffs_prev',
    'years_experience',
    'career_total_awards', 'career_most_valuable_player'
]

# Verify features exist
print("Feature verification:")
for name, features, df in [
    ('MVP', features_mvp, train_df),
    ('DPOY', features_dpoy, train_df),
    ('MIP', features_mip, train_df),
    ('FMVP', features_fmvp, train_df),
    ('ROY', features_roy, roy_train_df),
    ('COTY', features_coty, coty_train_df),
    ('ASG_MVP', features_asg_mvp, train_df)
]:
    missing = [f for f in features if f not in df.columns]
    if missing:
        print(f"⚠️ {name} missing: {missing}")
    else:
        print(f"✅ {name}: {len(features)} features OK")

Feature verification:
✅ MVP: 25 features OK
✅ DPOY: 15 features OK
✅ MIP: 15 features OK
✅ FMVP: 19 features OK
✅ ROY: 12 features OK
✅ COTY: 12 features OK
✅ ASG_MVP: 24 features OK


In [15]:
# ===============================================================
# (12) Train MVP Model
# ===============================================================
print("\n" + "="*70)
print("TRAINING MVP MODEL")
print("="*70)

target_mvp = 'won_most_valuable_player'
train_mvp = train_df.dropna(subset=features_mvp)

X_mvp = train_mvp[features_mvp]
y_mvp = train_mvp[target_mvp]

print(f"Training samples: {len(X_mvp)}")
print(f"MVP winners: {y_mvp.sum()}")

model_mvp = RandomForestClassifier(
    n_estimators=200, max_depth=10, min_samples_split=5,
    class_weight='balanced', random_state=42, n_jobs=-1
)
model_mvp.fit(X_mvp, y_mvp)

with open(f"{MODEL_DIR}model_mvp.pkl", "wb") as f:
    pickle.dump(model_mvp, f)
print(f"✅ MVP model saved")


TRAINING MVP MODEL
Training samples: 1502
MVP winners: 7
✅ MVP model saved


In [16]:
# ===============================================================
# (13) Train DPOY Model
# ===============================================================
print("\n" + "="*70)
print("TRAINING DPOY MODEL")
print("="*70)

target_dpoy = 'won_defensive_player_of_the_year'
train_dpoy = train_df.dropna(subset=features_dpoy)

X_dpoy = train_dpoy[features_dpoy]
y_dpoy = train_dpoy[target_dpoy]

print(f"Training samples: {len(X_dpoy)}")
print(f"DPOY winners: {y_dpoy.sum()}")

model_dpoy = RandomForestClassifier(
    n_estimators=200, max_depth=10,
    class_weight='balanced', random_state=42, n_jobs=-1
)
model_dpoy.fit(X_dpoy, y_dpoy)

with open(f"{MODEL_DIR}model_dpoy.pkl", "wb") as f:
    pickle.dump(model_dpoy, f)
print(f"DPOY model saved")


TRAINING DPOY MODEL
Training samples: 1502
DPOY winners: 7
DPOY model saved


In [17]:
# ===============================================================
# (14) Train MIP Model
# ===============================================================
print("\n" + "="*70)
print("TRAINING MIP MODEL")
print("="*70)

target_mip = 'won_most_improved_player'
train_mip = train_df[
    (train_df['years_experience'] >= 0)
].copy()
train_mip = train_mip.dropna(subset=features_mip)

X_mip = train_mip[features_mip]
y_mip = train_mip[target_mip]

print(f"Training samples: {len(X_mip)}")
print(f"MIP winners: {y_mip.sum()}")

model_mip = RandomForestClassifier(
    n_estimators=300, max_depth=8, min_samples_leaf=2,
    class_weight='balanced', random_state=42, n_jobs=-1
)
model_mip.fit(X_mip, y_mip)

with open(f"{MODEL_DIR}model_mip.pkl", "wb") as f:
    pickle.dump(model_mip, f)
print(f"MIP model saved")


TRAINING MIP MODEL
Training samples: 1502
MIP winners: 8
MIP model saved


In [18]:
# ===============================================================
# (15) Train FMVP Model
# ===============================================================
print("\n" + "="*70)
print("TRAINING FMVP MODEL")
print("="*70)

target_fmvp = 'won_wnba_finals_most_valuable_player'
train_fmvp = train_df[
    (train_df['made_playoffs_prev'] == 1) &
    (train_df['PostGP_prev'] > 0)
].copy()

available_fmvp = [f for f in features_fmvp if f in train_fmvp.columns]
train_fmvp = train_fmvp.dropna(subset=available_fmvp)

X_fmvp = train_fmvp[available_fmvp].fillna(0)
y_fmvp = train_fmvp[target_fmvp]

print(f"Training samples: {len(X_fmvp)}")
print(f"FMVP winners: {y_fmvp.sum()}")

if y_fmvp.sum() > 0:
    model_fmvp = RandomForestClassifier(
        n_estimators=200, max_depth=8,
        class_weight='balanced', random_state=42, n_jobs=-1
    )
    model_fmvp.fit(X_fmvp, y_fmvp)
    with open(f"{MODEL_DIR}model_fmvp.pkl", "wb") as f:
        pickle.dump(model_fmvp, f)
    print(f"FMVP model saved")
    features_fmvp = available_fmvp
else:
    print("No FMVP winners - skipping")


TRAINING FMVP MODEL
Training samples: 716
FMVP winners: 6
FMVP model saved


In [19]:
# ===============================================================
# (16) Train ROY Model
# ===============================================================
print("\n" + "="*70)
print("TRAINING ROY MODEL")
print("="*70)

target_roy = 'won_rookie_of_the_year'
train_roy = roy_train_df.dropna(subset=features_roy)

X_roy = train_roy[features_roy]
y_roy = train_roy[target_roy]

print(f"Training samples: {len(X_roy)}")
print(f"ROY winners: {y_roy.sum()}")

model_roy = RandomForestClassifier(
    n_estimators=200, max_depth=8, min_samples_split=5,
    class_weight='balanced', random_state=42, n_jobs=-1
)
model_roy.fit(X_roy, y_roy)

with open(f"{MODEL_DIR}model_roy.pkl", "wb") as f:
    pickle.dump(model_roy, f)
print(f"ROY model saved")


TRAINING ROY MODEL
Training samples: 564
ROY winners: 10
ROY model saved


In [20]:
# ===============================================================
# (17) Train COTY Model
# ===============================================================
print("\n" + "="*70)
print("TRAINING COTY MODEL")
print("="*70)

target_coty = 'won_coach_of_the_year'
train_coty = coty_train_df.dropna(subset=features_coty)

X_coty = train_coty[features_coty]
y_coty = train_coty[target_coty]

print(f"Training samples: {len(X_coty)}")
print(f"COTY winners: {y_coty.sum()}")

model_coty = RandomForestClassifier(
    n_estimators=200, max_depth=8, min_samples_split=5,
    class_weight='balanced', random_state=42, n_jobs=-1
)
model_coty.fit(X_coty, y_coty)

with open(f"{MODEL_DIR}model_coty.pkl", "wb") as f:
    pickle.dump(model_coty, f)
print(f"COTY model saved")


TRAINING COTY MODEL
Training samples: 145
COTY winners: 9
COTY model saved


In [21]:
# ===============================================================
# (18) Train ASG_MVP Model
# ===============================================================
print("\n" + "="*70)
print("TRAINING ASG_MVP MODEL")
print("="*70)
print("Note: Filtering to elite players only (top 10% by rating) as proxy for All-Star selection")

target_asg_mvp = 'won_all-star_game_most_valuable_player'

# Filter to elite players only
train_asg = train_df[train_df['is_elite_prev'] == 1].copy()
train_asg = train_asg.dropna(subset=features_asg_mvp)

X_asg = train_asg[features_asg_mvp]
y_asg = train_asg[target_asg_mvp]

print(f"Training samples (elite players): {len(X_asg)}")
print(f"ASG_MVP winners: {y_asg.sum()}")

if y_asg.sum() > 0:
    model_asg_mvp = RandomForestClassifier(
        n_estimators=200, max_depth=8, min_samples_split=5,
        class_weight='balanced', random_state=42, n_jobs=-1
    )
    model_asg_mvp.fit(X_asg, y_asg)
    
    with open(f"{MODEL_DIR}model_asg_mvp.pkl", "wb") as f:
        pickle.dump(model_asg_mvp, f)
    print(f"ASG_MVP model saved")
else:
    print("No ASG_MVP winners in training data - skipping")


TRAINING ASG_MVP MODEL
Note: Filtering to elite players only (top 10% by rating) as proxy for All-Star selection
Training samples (elite players): 149
ASG_MVP winners: 4
ASG_MVP model saved


In [22]:
# ===============================================================
# (18) Save feature sets
# ===============================================================
feature_sets = {
    'mvp': features_mvp,
    'dpoy': features_dpoy,
    'mip': features_mip,
    'fmvp': features_fmvp,
    'roy': features_roy,
    'coty': features_coty,
    'asg_mvp': features_asg_mvp,
    'main_awards': main_awards,
    'coach_award': coach_award
}

with open(f"{MODEL_DIR}feature_sets.pkl", "wb") as f:
    pickle.dump(feature_sets, f)

print("\n" + "="*70)
print("✅ ALL MODELS SAVED")
print("="*70)
for f in sorted(os.listdir(MODEL_DIR)):
    print(f"  - {f}")


✅ ALL MODELS SAVED
  - award_train_data.csv
  - coty_train_data.csv
  - elite_cutoffs.csv
  - feature_sets.pkl
  - league_averages.csv
  - model_asg_mvp.pkl
  - model_coty.pkl
  - model_dpoy.pkl
  - model_fmvp.pkl
  - model_mip.pkl
  - model_mvp.pkl
  - model_roy.pkl
  - predictions_asg_mvp_test.csv
  - predictions_coty_test.csv
  - predictions_roy_test.csv
  - predictions_summary.csv
  - predictions_test.csv
  - roy_train_data.csv


In [23]:
# ===============================================================
# (17) Cross-validation
# ===============================================================
print("\n" + "="*70)
print("CROSS-VALIDATION (Leave-One-Year-Out)")
print("="*70)

years = sorted(train_df['target_year'].unique())
results = []

for test_year in years[1:]:
    train_mask = train_df['target_year'] < test_year
    test_mask = train_df['target_year'] == test_year
    
    train_cv = train_df[train_mask].dropna(subset=features_mvp)
    test_cv = train_df[test_mask].dropna(subset=features_mvp)
    
    if len(train_cv) == 0 or len(test_cv) == 0:
        continue
    
    X_tr, y_tr = train_cv[features_mvp], train_cv[target_mvp]
    X_te, y_te = test_cv[features_mvp], test_cv[target_mvp]
    
    model_cv = RandomForestClassifier(
        n_estimators=200, max_depth=10, min_samples_split=5,
        class_weight='balanced', random_state=42, n_jobs=-1
    )
    model_cv.fit(X_tr, y_tr)
    
    proba = model_cv.predict_proba(X_te)[:, 1]
    test_cv = test_cv.copy()
    test_cv['mvp_proba'] = proba
    test_cv['mvp_rank'] = test_cv['mvp_proba'].rank(ascending=False, method='first')
    
    actual_winner = test_cv[test_cv[target_mvp] == 1]
    if len(actual_winner) > 0:
        winner_rank = actual_winner['mvp_rank'].values[0]
        results.append({'year': test_year, 'winner_rank': winner_rank})

if results:
    results_df = pd.DataFrame(results)
    print("\nMVP Model Results:")
    print(results_df.to_string(index=False))
    print(f"\nAverage Winner Rank: {results_df['winner_rank'].mean():.1f}")
    print(f"Top-3 Accuracy: {(results_df['winner_rank'] <= 3).mean()*100:.1f}%")


CROSS-VALIDATION (Leave-One-Year-Out)

MVP Model Results:
 year  winner_rank
    4          4.0
    5          1.0
    6          5.0
    7          1.0
    8          3.0
   10         10.0

Average Winner Rank: 4.0
Top-3 Accuracy: 50.0%


In [24]:
# ===============================================================
# (19) Summary
# ===============================================================
print("\n" + "="*70)
print("TRAINING COMPLETE")
print("="*70)

print(f"\n📊 Approach:")
print(f"   Predict awards in year Y using stats from year Y-1")
print(f"   All features are T-1 (previous year) based")
print(f"   ROY uses biometrics, college, and coach (no T-1 stats for rookies)")
print(f"   Predictions can be made BEFORE season starts")

print(f"\n🤖 Models:")
print(f"   - MVP: {len(features_mvp)} features")
print(f"   - DPOY: {len(features_dpoy)} features")
print(f"   - MIP: {len(features_mip)} features")
print(f"   - FMVP: {len(features_fmvp)} features")
print(f"   - ROY: {len(features_roy)} features (biometrics, college, coach)")
print(f"   - COTY: {len(features_coty)} features (team stats, coach history)")
print(f"   - ASG_MVP: {len(features_asg_mvp)} features (elite players only)")

print(f"\nReady")


TRAINING COMPLETE

📊 Approach:
   Predict awards in year Y using stats from year Y-1
   All features are T-1 (previous year) based
   ROY uses biometrics, college, and coach (no T-1 stats for rookies)
   Predictions can be made BEFORE season starts

🤖 Models:
   - MVP: 25 features
   - DPOY: 15 features
   - MIP: 15 features
   - FMVP: 19 features
   - ROY: 12 features (biometrics, college, coach)
   - COTY: 12 features (team stats, coach history)
   - ASG_MVP: 24 features (elite players only)

Ready
